In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pointbiserialr
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score,mean_absolute_percentage_error,mean_squared_error,mean_absolute_percentage_error
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,RobustScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,StratifiedKFold,LeaveOneOut,LeavePOut,validation_curve,learning_curve
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,StackingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
# from xgboost import XGBRegressor,XGBRFRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge,ARDRegression,Lasso, Ridge

## Cleaning the dataset for Thermal Conductivity 

In [ ]:
df_raw=pd.read_csv('sample10.csv')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Remove 'Sinter Method' column and row before calculating correlation matrix
df_filtered = df_raw.drop(columns=['Sinter method'], errors='ignore')
corr_matrix = df_filtered.corr()

# Set up the matplotlib figure
plt.figure(figsize=(20, 16))

# Rename specific labels with bold, subscripts, and Greek letters using LaTeX formatting
corr_matrix = corr_matrix.rename(
    index={'Relative Density': r'$\mathbf{\rho_{\mathbf{relative}}}$', 
           'Thermal Conductivity': r'$\mathbf{\sigma_{\mathbf{t}}}$', 
           'SiC': r'$\mathbf{C_{\mathbf{SiC}}}$', 
           'ZrB2': r'$\mathbf{C_{\mathbf{ZrB2}}}$', 
           'Sinter temp': r'$\mathbf{T_{\mathbf{sinter}}}$'},
    columns={'Relative Density': r'$\mathbf{\rho_{\mathbf{relative}}}$', 
             'Thermal Conductivity': r'$\mathbf{\sigma_{\mathbf{t}}}$', 
             'SiC': r'$\mathbf{C_{\mathbf{SiC}}}$', 
             'ZrB2': r'$\mathbf{C_{\mathbf{ZrB2}}}$', 
             'Sinter temp': r'$\mathbf{T_{\mathbf{sinter}}}$'}
)

# Draw the heatmap with increased font size for annotations and labels
sns.heatmap(corr_matrix, annot=True, cmap='plasma', vmin=-1, vmax=1,
            annot_kws={"size": 28,"weight": "bold"},  # Annotation font size
            cbar_kws={"shrink": 0.8})  # Color bar size 
# Customize the x and y labels with larger font
cbar = plt.gca().collections[0].colorbar
for label in cbar.ax.get_yticklabels():
    label.set_fontsize(24)
    label.set_fontweight('bold')


plt.xticks(fontsize=32, weight='bold')
plt.yticks(fontsize=32, weight='bold')

# Add a title with increased font size
plt.title('Pearson Correlation Matrix', fontsize=32,weight='bold')

# Display the plot
plt.show()

# Save the figure with increased label size
plt.savefig("Pearson_Correlation_Matrix_TC.png", format='png', bbox_inches='tight')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming filtered_df is your DataFrame
x = df_raw[['ZrB2', 'SiC', 'Relative Density',
       'Temp']]
y = df_raw[['Thermal Conductivity']]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(x_train, y_train)

# Predict on the testing data
y_pred = model.predict(x_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming filtered_df is your DataFrame
x = df_raw[['ZrB2', 'SiC', 'Relative Density',
       'Temp']]
y = df_raw['Thermal Conductivity']  # Changed to Series

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(x_train, y_train)

# Predict on the training and testing data
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Calculate the Mean Squared Error (MSE) for training and testing data
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Training Mean Squared Error:", train_mse)
print("Testing Mean Squared Error:", test_mse)

# Calculate the R² score for training and testing data
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
print("Training R^2 Score:", train_r2)
print("Testing R^2 Score:", test_r2)


In [ ]:
import pandas as pd


conditions = [
    (df_raw['Temp'] <= 100),
    (df_raw['Temp'] > 100) & (df_raw['Temp'] <= 500),
    (df_raw['Temp'] > 500) & (df_raw['Temp'] <= 1000),
    (df_raw['Temp'] > 1000) & (df_raw['Temp'] <= 1500),
    (df_raw['Temp'] > 1500)
]

labels = [0, 1,2, 3, 4]

# Use np.select to assign the labels based on the conditions
df_raw['Label'] = pd.Series(pd.NA, index=df_raw.index)  # Create an empty column 'Label'

df_raw['Label'] = pd.Series(
    pd.NA, dtype=pd.Int64Dtype(), index=df_raw.index
) 


df_raw['Label'] = np.select(conditions, labels, default=df_raw['Label'])

# Show the updated DataFrame
df_raw

In [ ]:
df_raw.groupby(['Label']).count()

In [ ]:
# Remove rows with missing 'Label' values
df_raw = df_raw.dropna(subset=['Label'])

# Convert 'Label' column to integers
df_raw['Label'] = df_raw['Label'].astype(int)


In [ ]:
df_raw

## Applying Synthetic Minority Oversampling Technique

In [ ]:
from imblearn.over_sampling import ADASYN, SMOTE, BorderlineSMOTE,SMOTENC,SVMSMOTE,KMeansSMOTE
ada = SVMSMOTE(random_state = 44, k_neighbors = 2)
X_res, y_res = ada.fit_resample(df_raw.iloc[:,:-1], df_raw.iloc[:,-1])

Ada_data = pd.concat([X_res, y_res], axis=1)
Ada_data = Ada_data.drop_duplicates()
Ada_data

In [ ]:
Ada_data.groupby(['Label']).count()


In [ ]:
X1 = Ada_data[['ZrB2', 'SiC', 'Sinter temp', 'Relative Density',"Temp"]]
Y1 = Ada_data['Thermal Conductivity']

In [ ]:
print(Ada_data.columns)

In [ ]:
df_raw = Ada_data
df_raw=df_raw.drop(['Label'],axis=1)
train_dataset = df_raw.sample(frac=0.8, random_state=44)
test_dataset = df_raw.drop(train_dataset.index)
test_dataset.info()

In [ ]:
y_train = train_dataset.pop('Thermal Conductivity') #y_train
y_test = test_dataset.pop('Thermal Conductivity')   # y_test

## Trying ML different models

In [ ]:
from sklearn.impute import SimpleImputer,KNNImputer

imptrain = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imptest = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

Train_scaler = MinMaxScaler()
Test_scaler = MinMaxScaler()


X_train_scaled = Train_scaler.fit_transform(train_dataset)
X_test_scaled = Test_scaler.fit_transform(test_dataset)

imptrain.fit(X_train_scaled)
imptest.fit(X_test_scaled)

X_train_scaled = imptrain.transform(X_train_scaled)
X_test_scaled = imptest.transform(X_test_scaled)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

def make_regression(X_train, y_train, X_test, y_test, model, model_name, verbose=True):
    """Apply selected regression model to data and measure error"""
    model.fit(X_train, y_train)
    
    # Predict on the training set
    y_train_pred = model.predict(X_train)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    # Predict on the testing set
    y_test_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    if verbose:
        
        #print("Train MAE = {:.4f} in {}".format(train_mae, model_name))
        # print("Train R^2 = {:.4f} in {}".format(train_r2, model_name))
        #print("Test MAE = {:.4f} in {}".format(test_mae, model_name))
        print(".".format(test_r2, model_name))
        
    return model, y_test_pred, train_mae, train_r2, test_mae, test_r2

# Assuming you have a dictionary of regression models
regression_models = {
    "DecisionTree": DecisionTreeRegressor(random_state=44),
    "SVR": SVR(),
    "RandomForest": RandomForestRegressor(random_state=44),
    "XGBoost": XGBRegressor(random_state=44),
    "KRR": KernelRidge(),
    "ABR": AdaBoostRegressor(random_state=44),
    "GBR": GradientBoostingRegressor(random_state=44),
    "ETR": ExtraTreesRegressor(random_state=44),
    "KNN": KNeighborsRegressor()
}

# Iterate over the dictionary of models
for model_name, model in regression_models.items():
    _, y_test_pred, train_mae, train_r2, test_mae, test_r2 = make_regression(X_train_scaled, y_train, X_test_scaled, y_test, model, model_name, verbose=True)
    print("R2 of", model_name, "=", test_r2)
    print("MAE of", model_name, "=", test_mae)
    print("Train MAE of", model_name, "=", train_mae)
    print("Train R2 of", model_name, "=", train_r2)


In [ ]:
def make_regression(X_train, y_train, X_test, y_test, model, model_name, verbose=True):
    model.fit(X_train,y_train)
    y_predict=model.predict(X_train)
    train_error = mean_absolute_error(y_train, y_predict)
    y_predict=model.predict(X_test)
    test_error = mean_absolute_error(y_test, y_predict)
    
    if verbose:
        print("Train error = "'{}'.format(train_error)+" in " + model_name)
        print("Test error = "'{}'.format(test_error)+" in " + model_name)
    trained_model = model
    
    return trained_model, y_predict, train_error, test_error

In [ ]:
regression_models = {
    "DecisionTree" : DecisionTreeRegressor(random_state=44),
    "SVR": SVR(),
    "RandomForest" : RandomForestRegressor(random_state=44),
    "XGBoost": XGBRegressor(random_state=44),
    "KRR":KernelRidge(),
    "ABR": AdaBoostRegressor(random_state=44),
    "GBR": GradientBoostingRegressor(random_state=44),
    "ETR": ExtraTreesRegressor(random_state=44),
    "KNN":KNeighborsRegressor()
}
for model_name in regression_models.keys():
    _, y_predict, _, _= make_regression(X_train_scaled, y_train, X_test_scaled, y_test,regression_models[model_name], model_name, verbose=True)
    print("R2 of",model_name,"=", r2_score(y_test,y_predict),"\n"),
    print("MAE of",model_name,"=", mean_absolute_error(y_test,y_predict),"\n")

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Initialize the Extra Trees Regressor
extra_trees = RandomForestRegressor(random_state=42)

# Train the model
extra_trees.fit(X_train, y_train)

# Predict on the test set
y_pred = extra_trees.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

## Final model for Thermal conductivity

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Initialize the Extra Trees Regressor
extra_trees = ExtraTreesRegressor(n_estimators=100, random_state=42)

# Train the model
extra_trees.fit(X_train, y_train)

# Predict on the test set
y_pred_test = extra_trees.predict(X_test)
y_pred_train = extra_trees.predict(X_train)

# Calculate Mean Squared Error for testing set
mse_test = mean_squared_error(y_test, y_pred_test)
print("Mean Squared Error (Test):", mse_test)

# Calculate Mean Absolute Error for testing set
mae_test = mean_absolute_error(y_test, y_pred_test)
print("Mean Absolute Error (Test):", mae_test)

# Calculate R² score for testing set
r2_test = r2_score(y_test, y_pred_test)
print("R² Score (Test):", r2_test)

# Calculate Mean Absolute Error for training set
mae_train = mean_absolute_error(y_train, y_pred_train)
print("Mean Absolute Error (Train):", mae_train)

# Calculate R² score for training set
r2_train = r2_score(y_train, y_pred_train)
print("R² Score (Train):", r2_train)


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd

# Assuming X1 and Y1 are already defined as your features and target respectively

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Initialize the SVR model
svr = SVR()

# Define the hyperparameter grid for SVR
param_grid = {
    'kernel': ['linear', 'rbf'],  # I removed 'poly' and 'sigmoid' for now to simplify testing
    'C': [0.1, 1, 10],  # Regularization parameter
    'epsilon': [0.01, 0.1, 0.5],  # Specifies the epsilon-tube
    'gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf' and 'linear'
}

# Initialize GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model on the test set
mse = mean_squared_error(y_test, y_pred)
r2_test = r2_score(y_test, y_pred)
print(f"Mean Squared Error (Test): {mse}")
print(f"R^2 Score (Test): {r2_test}")

# Evaluate the model on the training set
r2_train = best_model.score(X_train, y_train)
print(f"R^2 Score (Train): {r2_train}")

# Output the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Plotting True vs Predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.title('True vs Predicted Values')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')

# Add a diagonal reference line (Perfect Prediction line)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')

# Add grid for better readability
plt.grid(True)

# Display the plot
plt.show()

# Save the plot as a PNG file
plt.savefig("True_vs_Predicted_Values_SVR.png", format='png')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define the number of folds for cross-validation
num_folds = 5

# Initialize the KFold cross-validation splitter
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics for each fold
mse_scores = []
mae_scores = []
r2_scores = []

# Iterate over each fold
for fold_idx, (train_index, val_index) in enumerate(kf.split(X1)):
    print(f"Fold {fold_idx + 1}/{num_folds}")
    
    # Split data into training and validation sets for this fold
    X_train, X_val = X1.iloc[train_index], X1.iloc[val_index]
    y_train, y_val = Y1.iloc[train_index], Y1.iloc[val_index]
   
    # Make predictions on the validation set
    y_pred = best_model.predict(X_val)
    
    # Calculate evaluation metrics for this fold
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    # Append scores to lists
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)
    
    # Print evaluation metrics for this fold
    print(f"  Validation R^2: {r2:.4f}")
    print(f"  Validation MSE: {mse:.4f}")
    print(f"  Validation MAE: {mae:.4f}")

# Calculate the average scores across all folds
average_mse = np.mean(mse_scores)
average_mae = np.mean(mae_scores)
average_r2 = np.mean(r2_scores)

print(f"\nAverage Validation MSE: {average_mse:.4f}")
print(f"Average Validation MAE: {average_mae:.4f}")
print(f"Average Validation R^2: {average_r2:.4f}")


In [ ]:
import shap
explainer = shap.Explainer(best_model, X_train)

# Calculate SHAP values
shap_values = explainer(X_train)

# Summary plot (global interpretation)
shap.summary_plot(shap_values, X_train)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming X1 is your dataset and best_model is your trained model
importances = best_model.feature_importances_
features = pd.DataFrame({'Feature': X1.columns, 'Importance': importances})

# Sort features by importance
features = features.sort_values(by='Importance', ascending=False)

# Plotting feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=features, orient='h', palette='viridis')
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()
plt.savefig("Feature_Importances_TC.png", format='png')


In [ ]:
importances = best_model.feature_importances_
features = pd.DataFrame({'Feature': X1.columns, 'Importance': importances})

# Sort features by importance
features = features.sort_values(by='Importance', ascending=False)

# Plotting feature importances using a swarm plot
plt.figure(figsize=(10, 6))
sns.swarmplot(x='Importance', y='Feature', data=features, palette='viridis')
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()
plt.savefig("Swarn_Plot_TC.png", format='png')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define the number of folds for cross-validation
num_folds = 5

# Initialize the KFold cross-validation splitter
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics for each fold
mse_scores = []
train_r2_scores = []
val_r2_scores = []

# Iterate over each fold
for fold_idx, (train_index, val_index) in enumerate(kf.split(X1)):
    print(f"Fold {fold_idx + 1}/{num_folds}")
    
    # Split data into training and validation sets for this fold
    X_train, X_val = X1.iloc[train_index], X1.iloc[val_index]
    y_train, y_val = Y1.iloc[train_index], Y1.iloc[val_index]
    
    # Fit the model to the training data
    best_model.fit(X_train, y_train)

    
    # Calculate R² score for training data
    y_pred_train = best_model.predict(X_train)
    train_r2 = r2_score(y_train, y_pred_train)
    
    train_r2_scores.append(train_r2)
   
    
    print(f"  Training R^2: {train_r2:.4f}")
    



# Calculate the average R² score across all folds for training and validation data
average_train_r2 = np.mean(train_r2_scores)
print(f"\nAverage Training R²: {average_train_r2:.4f}")



## Applying Model trained for thermal conductivity on cleaned oxide scale thickness dataset

In [ ]:
dataframe=pd.read_csv('thermal_conductivity_vs_SiC_concentration_multiple_10.csv')
print(dataframe.columns)

In [ ]:
dataframe

In [ ]:
X_testing = dataframe[['ZrB2', 'SiC', 'Sinter temp', 'Relative Density', 'Temp']]

In [ ]:
y_prediction = best_model.predict(X_testing)
dataframe["Thermal Conductivity"] = y_prediction
dataframe.to_csv('thermal_conductivity_vs_SiC_concentration.csv', index=False)

In [ ]:
dataframe

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis
plt.figure(figsize=(8, 6))

# Scatter plot for the first segment (0-14)
plt.scatter(dataframe['SiC'][0:15], dataframe['Thermal Conductivity'][0:15], color='yellow', alpha=0.5, label='2000')  

# Scatter plot for the second segment (15-29)
plt.scatter(dataframe['SiC'][15:30], dataframe['Thermal Conductivity'][15:30], color='red', alpha=0.5, label='1800')

# Scatter plot for the third segment (30-44)
plt.scatter(dataframe['SiC'][30:45], dataframe['Thermal Conductivity'][30:45], color='black', alpha=0.5, label='1600')

# Scatter plot for the fourth segment (45-59)
plt.scatter(dataframe['SiC'][45:60], dataframe['Thermal Conductivity'][45:60], color='grey', alpha=0.5, label='1400')

# Add title and labels
plt.title('SiC vs Thermal Conductivity at 1900 Celsius and 100 Percent Densification')  
plt.xlabel('SiC Percent in ZrB2 Composite (%)') 
plt.ylabel('Thermal Conductivity (W/mK)')  

# Add grid
plt.grid(True) 

# Add legend to distinguish the segments
plt.legend()

# Adjust layout
plt.tight_layout()  

# Save the plot
plt.savefig('SiC_conc_vs_Thermal_Conductivity.png')

# Show the plot
plt.show()


In [ ]:
print(dataframe.iloc[59])

In [ ]:
Dataframe=pd.read_csv("Completed_maybe.csv")
print(Dataframe.columns)

In [ ]:
Dataframe
Dataframe = Dataframe.dropna()


In [ ]:
X_testin = Dataframe[['ZrB2', 'SiC', 'Sinter temp', 'Relative Density', 'Temp']]

In [ ]:
y_predictio = best_model.predict(X_testin)


In [ ]:
Dataframe["Thermal Conductivity"] = y_predictio
Dataframe
Dataframe.to_csv('Oxidation_ZrB2.csv', index=False)

In [ ]:
Dataframe